In [3]:
import os
import opensmile
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from natsort import natsorted
import pandas as pd
from pydub import AudioSegment as AS
from ChaFile import *

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

dir_healthy_wav = input("enter the (full) path to your directory of healthy WAV files --> ")
dir_healthy_cha = input("enter the (full) path to your directory of healthy CHA files --> ")

dir_AD_wav = input("enter the (full) path to your directory of AD WAV files --> ")
dir_AD_cha = input("enter the (full) path to your directory of AD CHA files --> ")

dir_MCI_wav = input("enter the (full) path to your directory of MCI WAV files --> ")
dir_MCI_cha = input("enter the (full) path to your directory of MCI CHA files --> ")


In [4]:
def checker(wav, cha):
    files_from_dir = os.listdir(wav)
    files_from_dir_2 = os.listdir(cha)
    if len(files_from_dir) != len(files_from_dir_2):
        quit("not equal")

checker(dir_healthy_wav, dir_healthy_cha)
checker(dir_AD_wav, dir_AD_cha)
checker(dir_MCI_wav, dir_MCI_cha)

AD_cha_files = natsorted(os.listdir(dir_AD_cha))
AD_wav_files = natsorted(os.listdir(dir_AD_wav))

MCI_cha_files = natsorted(os.listdir(dir_MCI_cha))
MCI_wav_files = natsorted(os.listdir(dir_MCI_wav))

Healthy_cha_files = natsorted(os.listdir(dir_healthy_cha))
Healthy_wav_files = natsorted(os.listdir(dir_healthy_wav))

In [36]:
keys_1 = ["command_1", "command_2", "command_3", "command_4", "command_5", "command_6", "command_7", "command_8", "command_9", 
"command_10", "command_11", "command_12", "command_13", "command_14", "command_15", "command_16", "command_17", "command_18", "command_19",
"command_20", "command_21", "command_22", "command_23", "command_24", "command_25", "command_26", "command_27", "command_28", "command_29",
"command_30", "command_31", "command_32", "command_33", "RESPONSE COMMAND"]
keywords = ["weather", "date", "time", "Thanksgiving", ["chocolate", "chip", "cookies"], ["two", "four"], "tablespoons", "symptom", "classical",
["volume", "eight"], "jazz", ["volume", "six"], "stop", "laundry", "dog", "reminders", "birthday", 
["five", "seconds"], "alarm", ["oranges", "grapes"], ["what", "is", "in", "shopping"], "call", ["hang", "up"], "find", "quit", ["bedroom", "light", "on"],
"red", "brightness", ["bedroom", "light", "off"], ["open", "kitchen", "camera"], "hide", ["white", "collar"], "pause", "yes"]

print(len(keys_1))
print(len(keywords))

dz = dict(zip(keys_1, keywords))


34
34


In [43]:
def categorize(cha_dir, wav_dir, cha_file, wav_file):
    os.chdir(cha_dir)
    cha = ChaFile(cha_file)
    lines = cha.getLines()
    data_set = []
    participant = cha_file
    unmatched_count = 1
    for data in lines:
        action_taken = 0
        utterance = data["xol"]
        timestamp = data["bullet"]
        utterance = utterance.split(" ")
        print(utterance)
        if utterance[0][0] == "[":
                print("FLAG")
                keys = "UNRECOGNIZED COMMAND"
                data_set.append([participant, keys, 0, utterance, timestamp])
        else:        
            for word in dz.values():
                if type(word) is list:
                    key_list = []
                    c = all(x in utterance for x in word)
                    if c:
                        action_taken += 1
                        keys = [k for k, v in dz.items() if v == word]
                        key_list.append(keys)
                        print("matched", word, keys, timestamp)
                        data_set.append([participant, key_list, word, utterance, timestamp])
                if word in utterance:
                    action_taken = 1
                    keys = [k for k, v in dz.items() if v == word]
                    print("matched", word, keys, timestamp)
                    data_set.append([participant, keys, word, utterance, timestamp])
            if action_taken == 0:
                keys = ["UNMATCHED COMMAND #" + str(unmatched_count)] 
                print("FAILED TO MATCH")
                data_set.append([participant, keys, 0, utterance, timestamp])
                unmatched_count += 1
    frame = pd.DataFrame(data_set)
    print(frame)
    os.chdir("..")
    # with pd.ExcelWriter("output.xlsx", mode="a") as writer:
    #     frame.to_excel(writer, sheet_name=participant)

In [44]:
for i in range(len(AD_cha_files)):
    categorize(dir_AD_cha, dir_AD_wav, AD_cha_files[i], AD_wav_files[i])
for i in range(len(MCI_cha_files)):
    categorize(dir_MCI_cha, dir_MCI_wav, MCI_cha_files[i], MCI_wav_files[i])
for i in range(len(Healthy_cha_files)):
    categorize(dir_healthy_cha, dir_healthy_wav, Healthy_cha_files[i], Healthy_wav_files[i])

['Alexa', 'what', 'is', 'the', 'weather', 'outside', '?', '']
matched weather ['command_1'] [0, 3250]
['Alexa', 'what', 'is', "today's", 'date', '?', '']
matched date ['command_2'] [3250, 5750]
['Alexa', 'what', 'time', 'is', 'it', '?', '']
matched time ['command_3'] [5750, 8490]
['Alexa', 'when', 'is', 'Thanksgiving', '?', '']
matched Thanksgiving ['command_4'] [8490, 11680]
['Alexa', 'how', 'do', 'you', 'bake', 'chocolate', 'chip', 'cookies', '?', '']
matched ['chocolate', 'chip', 'cookies'] ['command_5'] [11680, 15530]
['Alexa', 'what', 'is', 'two', 'times', 'four', '?', '']
matched ['two', 'four'] ['command_6'] [15530, 18210]
['Alexa', 'how', 'many', 'tablespoons', 'in', 'a', 'cup', '?', '']
matched tablespoons ['command_7'] [18210, 21580]
['Alexa', 'how', 'do', 'you', 'spell', 'symptom', '?', '']
matched symptom ['command_8'] [21580, 24590]
['Alexa', 'play', 'classical', 'music', '.', '']
matched classical ['command_9'] [24590, 27780]
['Alexa', 'volume', 'to', 'eight', '.', '']
ma